#RAG Chatbot with LangChain

In this notebook I have built an AI chatbot from start-to-finish.I have used LangChain, OpenAI, and Pinecone vector DB, to build a chatbot capable of learning from the external world using Retrieval Augmented Generation (RAG).

I have used a dataset sourced from the Llama 2 ArXiv paper and other related papers to help the chatbot answer questions about the latest and greatest in the world of GenAI.

By the end of this notebook I have a functioning chatbot and RAG pipeline that can hold a conversation and provide informative responses based on a knowledge base.

I have installed some Python libraries. Here's a brief overview of what each library does:

langchain: This is a library for GenAI. I have used it to chain together different language models and components for the chatbot.

openai: This is the official OpenAI Python client. I have used it to interact with the OpenAI API and generate responses for the chatbot.

datasets: This library provides a vast array of datasets for machine learning. I have used it to load the knowledge base for the chatbot.

pinecone-client: This is the official Pinecone Python client. I have used it to interact with the Pinecone API and store the chatbot's knowledge base in a vector database.

In [ ]:
!pip install -qU \
    langchain==0.0.354 \
    openai==1.6.1 \
    datasets==2.15.1 \
    pinecone-client==3.1.0 \
    tiktoken==0.5.2

### Building a Chatbot (no RAG)

I have reiled on the LangChain library to bring together the different components needed for the chatbot. To begin, I have created a simple chatbot without any retrieval augmentation. I have done this by initializing a `ChatOpenAI` object. For this generated an [OpenAI API key](https://platform.openai.com/account/api-keys).

In [ ]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or "YOUR_API_KEY"

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

Chats with OpenAI's `gpt-3.5-turbo` and `gpt-4` chat models are structured (in plain text) like this:

```
System: You are a helpful assistant.

User: Hi AI, how are you today?

Assistant: I'm great thank you. How can I help you?

User: I'd like to understand string theory.

Assistant:
```

The final `"Assistant:"` without a response is what would prompt the model to continue the conversation. In the official OpenAI `ChatCompletion` endpoint these would be passed to the model in a format like:

```python
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi AI, how are you today?"},
    {"role": "assistant", "content": "I'm great thank you. How can I help you?"}
    {"role": "user", "content": "I'd like to understand string theory."}
]
```

In LangChain there is a slightly different format. We use three _message_ objects like so:

In [ ]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

The format is very similar,just swapped the role of `"user"` for `HumanMessage`, and the role of `"assistant"` for `AIMessage`.

We generate the next response from the AI by passing these messages to the `ChatOpenAI` object.

In [ ]:
res = chat(messages)
res

AIMessage(content='String theory is a theoretical framework in physics that aims to explain the fundamental nature of particles and their interactions. According to string theory, the basic building blocks of the universe are not particles, but tiny vibrating strings.\n\nHere are some key points to help you understand string theory:\n\n1. Fundamental Particles: In traditional particle physics, elementary particles are considered point-like entities. In string theory, they are replaced by tiny, one-dimensional strings. These strings can vibrate in different modes, producing different particle properties such as mass and charge.\n\n2. Extra Dimensions: String theory suggests that there may be extra dimensions beyond the three spatial dimensions (length, width, and height) that we experience. These additional dimensions are believed to be curled up or compactified at incredibly small scales, making them invisible to our current observations.\n\n3. Unification of Forces: String theory has 

In response I get another AI message object. I can print it more clearly like so:

In [ ]:
print(res.content)

String theory is a theoretical framework in physics that aims to explain the fundamental nature of particles and their interactions. According to string theory, the basic building blocks of the universe are not particles, but tiny vibrating strings.

Here are some key points to help you understand string theory:

1. Fundamental Particles: In traditional particle physics, elementary particles are considered point-like entities. In string theory, they are replaced by tiny, one-dimensional strings. These strings can vibrate in different modes, producing different particle properties such as mass and charge.

2. Extra Dimensions: String theory suggests that there may be extra dimensions beyond the three spatial dimensions (length, width, and height) that we experience. These additional dimensions are believed to be curled up or compactified at incredibly small scales, making them invisible to our current observations.

3. Unification of Forces: String theory has the potential to unify the 

Because `res` is just another `AIMessage` object, I can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [ ]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to produce a unified theory because it incorporates all the fundamental forces of nature within a single framework. The four fundamental forces in physics are gravity, electromagnetism, and the strong and weak nuclear forces. 

In the standard model of particle physics, these forces are described by different theories that are not fully compatible with each other. For example, gravity is described by general relativity, which explains the behavior of massive objects on large scales, while the other forces are described by quantum field theories.

String theory, on the other hand, provides a framework that can potentially reconcile and unify these forces. The vibrations of the tiny strings in string theory can give rise to different types of particles, and the way these particles interact can reproduce the behavior of the various fundamental forces. This means that, in principle, all the forces of nature could be described by a sin

### Dealing with Hallucinations

I have the chatbot, but the knowledge of LLMs can be limited. The reason for this is that LLMs learn all they know during training. An LLM essentially compresses the "world" as seen in the training data into the internal parameters of the model. We call this knowledge the _parametric knowledge_ of the model.

By default, LLMs have no access to the external world.

The result of this is very clear when we ask LLMs about more recent information, like about the new Llama 2 LLM.

In [ ]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What is so special about Llama 2?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [ ]:
print(res.content)

I'm sorry, but I don't have any specific information about "Llama 2." It could be a reference to something specific that I am not aware of. Could you please provide more context or clarify your question?


Our chatbot can no longer help us, it doesn't contain the information we need to answer the question. It was very clear from this answer that the LLM doesn't know the informaiton, but sometimes an LLM may respond like it _does_ know the answer — and this can be very hard to detect.

OpenAI have since adjusted the behavior for this particular example as we can see below:

In [ ]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Can you tell me about the LLMChain in LangChain?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [ ]:
print(res.content)

I apologize, but I couldn't find any specific information about "LLMChain" or "LangChain." It's possible that these terms are specific to a particular context or project that I am not familiar with. If you can provide more details or clarify your question, I'll do my best to assist you.


There is another way of feeding knowledge into LLMs. It is called _source knowledge_ and it refers to any information fed into the LLM via the prompt. We can try that with the LLMChain question. We can take a description of this object from the LangChain documentation.

In [ ]:
llmchain_information = [
    "A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.",
    "Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.",
    "LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with its environment. As such, the LangChain framework is designed with the objective in mind to enable those types of applications."
]

source_knowledge = "\n".join(llmchain_information)

We can feed this additional knowledge into our prompt with some instructions telling the LLM how we'd like it to use this information alongside our original query.

In [ ]:
query = "Can you tell me about the LLMChain in LangChain?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

Now we feed this into our chatbot as we were before.

In [ ]:
# create a new user prompt
prompt = HumanMessage(
    content=augmented_prompt
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [ ]:
print(res.content)

The LLMChain is a specific type of chain within the LangChain framework. It is designed to facilitate the development of applications powered by language models. 

A LLMChain consists of three main components: a PromptTemplate, a language model (either an LLM or a ChatModel), and an optional output parser. 

The PromptTemplate takes multiple input variables and formats them into a prompt. This prompt is then passed to the language model, which generates a response based on the provided input. 

The optional output parser, if provided, is used to parse the output of the language model into a final format that suits the specific application's needs. This allows for customization and flexibility in processing the model's output.

The LangChain framework, of which the LLMChain is a part, aims to enable powerful and differentiated applications by not only connecting to a language model via an API but also by being data-aware and agentic. Being data-aware means that the framework allows for 

The quality of this answer is phenomenal. This is made possible with the idea of augmented our query with external knowledge (source knowledge). There's just one problem — how do we get this information in the first place?
Pinecone and vector databases can help in this here.But first, i will need a dataset.



### Importing the Data

Here I will be importing the data. I will be using the Hugging Face Datasets library to load the data. Specifically, I will be using the `"jamescalam/llama-2-arxiv-papers"` dataset. This dataset contains a collection of ArXiv papers which will serve as the external knowledge base for the chatbot.

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "jamescalam/llama-2-arxiv-papers-chunked",
    split="train"
)

dataset

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 4838
})

In [ ]:
dataset[0]

{'doi': '1102.0183',
 'chunk-id': '0',
 'chunk': 'High-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nTechnical Report No. IDSIA-01-11\nJanuary 2011\nIDSIA / USI-SUPSI\nDalle Molle Institute for Arti\x0ccial Intelligence\nGalleria 2, 6928 Manno, Switzerland\nIDSIA is a joint institute of both University of Lugano (USI) and University of Applied Sciences of Southern Switzerland (SUPSI),\nand was founded in 1988 by the Dalle Molle Foundation which promoted quality of life.\nThis work was partially supported by the Swiss Commission for Technology and Innovation (CTI), Project n. 9688.1 IFF:\nIntelligent Fill in Form.arXiv:1102.0183v1  [cs.AI]  1 Feb 2011\nTechnical Report No. IDSIA-01-11 1\nHigh-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nJanuary 2011\nAbs

#### Dataset Overview

The dataset I am using is sourced from the Llama 2 ArXiv papers. It is a collection of academic papers from ArXiv, a repository of electronic preprints approved for publication after moderation. Each entry in the dataset represents a "chunk" of text from these papers.

Because most Large Language Models (LLMs) only contain knowledge of the world as it was during training, they cannot answer our questions about Llama 2 — at least not without this data.

### Task 4: Building the Knowledge Base

So, I now have a dataset that can serve as the chatbot knowledge base. My next task is to transform that dataset into the knowledge base that the chatbot can use. To do this I must use an embedding model and vector database.

I begin by initializing my connection to Pinecone, this requires a [free API key](https://app.pinecone.io).

In [ ]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY") or "YOUR_API_KEY"

# configure client
pc = Pinecone(api_key=api_key)

Now I setup my index specification, this allows me to define the cloud provider and region where I want to deploy my index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [ ]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

Then I initialize the index. I will be using OpenAI's `text-embedding-ada-002` model for creating the embeddings, so I have set the `dimension` to `1536`.

In [ ]:
import time

index_name = 'llama-2-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

My index is now ready but it's empty. It is a vector index, so it needs vectors. As mentioned, to create these vector embeddings I will OpenAI's `text-embedding-ada-002` model — we can access it via LangChain like so:

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

Using this model I have created embeddings below:

In [ ]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

From this I get two (aligning to the two chunks of text) 1536-dimensional embeddings.

I am now ready to embed and index all my data! I will do this by looping through the dataset and embedding and inserting everything in batches.

In [ ]:
from tqdm.auto import tqdm  # for progress bar

data = dataset.to_pandas()  # this makes it easier to iterate over the dataset

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [x['chunk'] for _, x in batch.iterrows()]
    # embed text
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'],
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

  0%|          | 0/49 [00:00<?, ?it/s]

 I have checked that the vector index has been populated using `describe_index_stats` like before:

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

#### Retrieval Augmented Generation

 I have built a fully-fledged knowledge base. Now it's time to connect that knowledge base to the chatbot. To do that I will be diving back into LangChain and reusing the template prompt from earlier.

To use LangChain here I need to load the LangChain abstraction for a vector index, called a `vectorstore`. I will pass in our vector `index` to initialize the object.

In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

/Users/jamesbriggs/opt/anaconda3/envs/ml/lib/python3.9/site-packages/langchain_community/vectorstores/pinecone.py:74: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Using this `vectorstore` I can already query the index and see if I have any relevant information given our question about Llama 2.

In [ ]:
query = "What is so special about Llama 2?"

vectorstore.similarity_search(query, k=3)

[Document(page_content='Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang\nRoss Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang\nAngela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic\nSergey Edunov Thomas Scialom\x03\nGenAI, Meta\nAbstract\nIn this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned\nlarge language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.\nOur ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our\nmodels outperform open-source chat models on most benchmarks we tested, and based on\nourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed description of our approach to ﬁne-tuning and safety', metadata={'source': 'http://arxiv.org/pdf/2307.09288', 'title': 'Llama 2: Open Foundation and Fine-Tun

We return a lot of text here and it's not that clear what we need or what is relevant. Fortunately, our LLM will be able to parse this information much faster than us. All I need is to connect the output from `vectorstore` to my `chat` chatbot. To do that I have used the same logic as we used earlier.

In [ ]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

Using this I produced an augmented prompt:

In [ ]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov Thomas Scialom
GenAI, Meta
Abstract
In this work, we develop and release Llama 2, a collection of pretrained and ﬁne-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our ﬁne-tuned LLMs, called L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc , are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
ourhumanevaluationsforhelpfulnessandsafety,maybeasuitablesubstituteforclosedsource models. We provide a detailed description of our approach to ﬁne-tuning and safety
asChatGPT,BARD,andClaude. TheseclosedproductLLMsareheavilyﬁne-tunedtoalignwith

There is still a lot of text here, so let's pass it onto our chat model to see how it performs.

In [ ]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) developed in a work. These LLMs range in scale from 7 billion to 70 billion parameters. They are optimized specifically for dialogue use cases.

What makes Llama 2 special is that its fine-tuned LLMs, such as L/l.sc/a.sc/m.sc/a.sc /two.taboldstyle-C/h.sc/a.sc/t.sc, have shown better performance compared to existing open-source chat models on various benchmarks. They have also been evaluated in terms of helpfulness and safety, and they appear to be on par with some closed-source models.

This work aims to provide a detailed description of the approach used for fine-tuning and ensuring safety in Llama 2, similar to other closed-source LLMs like ChatGPT, BARD, and Claude. The closed-source LLMs are extensively fine-tuned to align with human preferences, enhancing their usability and safety. However, this fine-tuning process often involves significant costs in terms of computation and human annotation, and it

We can continue with more Llama 2 questions. Let's try _without_ RAG first:

In [ ]:
prompt = HumanMessage(
    content="what safety measures were used in the development of llama 2?"
)

res = chat(messages + [prompt])
print(res.content)

In the provided context, it is mentioned that Llama 2, the collection of pretrained and fine-tuned large language models (LLMs), includes safety measures in its development. However, the specific safety measures used in the development of Llama 2 are not detailed in the given text. It only mentions that safety is a consideration in their approach to fine-tuning and that closed-source models are heavily fine-tuned to align with human preferences, enhancing their usability and safety.

To obtain more specific information regarding the safety measures implemented in the development of Llama 2, it would be necessary to refer to the original research paper or documentation related to Llama 2.


The chatbot is able to respond about Llama 2 thanks to it's conversational history stored in `messages`. However, it doesn't know anything about the safety measures themselves as I have not provided it with that information via the RAG pipeline. Let's try again but with RAG.

In [ ]:
prompt = HumanMessage(
    content=augment_prompt(
        "what safety measures were used in the development of llama 2?"
    )
)

res = chat(messages + [prompt])
print(res.content)

Based on the provided contexts, the development of Llama 2 involved several safety measures. The authors of Llama 2 took steps to increase the safety of the models by using safety-specific data annotation and tuning. They also conducted red-teaming and employed iterative evaluations to ensure safety. These measures were implemented to reduce potential risks and enhance the safety of the Llama 2 models.

The authors of Llama 2 also emphasized the importance of improving the safety of language models in their work. They provided a detailed description of their approach to fine-tuning and safety, aiming to contribute to the responsible development of large language models (LLMs). By sharing their methodology and observations, they hope to enable the community to reproduce fine-tuned LLMs and continue to enhance their safety.

It is worth noting that the specific details of the safety measures used in the development of Llama 2 may require further examination of the referenced papers and r

We get a much more informed response that includes several items missing in the previous non-RAG response, such as "red-teaming", "iterative evaluations", and the intention of the researchers to share this research to help "improve their safety, promoting responsible development in the field".